### Importing libraries

In [1]:
import pypots
import os
import sys
from pypots.utils.metrics import calc_mae
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS, MRNN, USGAN, GPVAE
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

2025-03-17 23:01:18.247549: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742263278.262716  171926 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742263278.267601  171926 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-17 23:01:18.283283: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/pypots/nn/functional/cuda.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is


████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



### Loading database

In [2]:
set_random_seed()

from pypotsModify.benchpotsMAE.datasets import preprocess_physionet2012

# Load the PhysioNet-2012 dataset
physionet2012_dataset = preprocess_physionet2012(subset="all", rate=0.1, normalization = 1)

# Take a look at the generated PhysioNet-2012 dataset, you'll find that everything has been prepared for you,
# data splitting, normalization, additional artificially-missing values for evaluation, etc.
print(physionet2012_dataset.keys())

2025-03-17 23:01:23 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-03-17 23:01:23 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-03-17 23:01:23 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-03-17 23:01:23 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-03-17 23:01:23 [INFO]: Loaded successfully!
2025-03-17 23:01:37 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-17 23:01:37 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-17 23:01:37 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [3]:
# assemble the datasets for training
dataset_for_training = {
    "X": physionet2012_dataset['train_X'],
}
# assemble the datasets for validation
dataset_for_validating = {
    "X": physionet2012_dataset['val_X'],
    "X_ori": physionet2012_dataset['val_X_ori'],
}

dataset_for_testing_ori = {
    "X_ori": physionet2012_dataset['test_X_ori'],
    "female_gender_test_X_ori": physionet2012_dataset['female_gender_test_X_ori'],
    "male_gender_test_X_ori": physionet2012_dataset['male_gender_test_X_ori'],
    "undefined_gender_test_X_ori": physionet2012_dataset['undefined_gender_test_X_ori'],
    "more_than_or_equal_to_65_test_X_ori":  physionet2012_dataset['more_than_or_equal_to_65_test_X_ori'],
    "less_than_65_test_X_ori": physionet2012_dataset['less_than_65_test_X_ori'],
    "ICUType_1_test_X_ori": physionet2012_dataset['ICUType_1_test_X_ori'],
    "ICUType_2_test_X_ori": physionet2012_dataset['ICUType_2_test_X_ori'],
    "ICUType_3_test_X_ori": physionet2012_dataset['ICUType_3_test_X_ori'],
    "ICUType_4_test_X_ori": physionet2012_dataset['ICUType_4_test_X_ori'],
    "classificacao_undefined_test_X_ori": physionet2012_dataset['classificacao_undefined_test_X_ori'],
    "classificacao_baixo_peso_test_X_ori": physionet2012_dataset['classificacao_baixo_peso_test_X_ori'],
    "classificacao_normal_peso_test_X_ori": physionet2012_dataset['classificacao_normal_peso_test_X_ori'],
    "classificacao_sobrepeso_test_X_ori": physionet2012_dataset['classificacao_sobrepeso_test_X_ori'],
    "classificacao_obesidade_1_test_X_ori": physionet2012_dataset['classificacao_obesidade_1_test_X_ori'],
    "classificacao_obesidade_2_test_X_ori": physionet2012_dataset['classificacao_obesidade_2_test_X_ori'],
    "classificacao_obesidade_3_test_X_ori": physionet2012_dataset['classificacao_obesidade_3_test_X_ori']
}

# assemble the datasets for test
dataset_for_testing = {
    "X": physionet2012_dataset['test_X'],
    "female_gender_test_X": physionet2012_dataset['female_gender_test_X'],
    "male_gender_test_X": physionet2012_dataset['male_gender_test_X'],
    "undefined_gender_test_X": physionet2012_dataset['undefined_gender_test_X'],
    "more_than_or_equal_to_65_test_X":  physionet2012_dataset['more_than_or_equal_to_65_test_X'],
    "less_than_65_test_X": physionet2012_dataset['less_than_65_test_X'],
    "ICUType_1_test_X": physionet2012_dataset['ICUType_1_test_X'],
    "ICUType_2_test_X": physionet2012_dataset['ICUType_2_test_X'],
    "ICUType_3_test_X": physionet2012_dataset['ICUType_3_test_X'],
    "ICUType_4_test_X": physionet2012_dataset['ICUType_4_test_X'],
    "classificacao_undefined_test_X": physionet2012_dataset['classificacao_undefined_test_X'],
    "classificacao_baixo_peso_test_X": physionet2012_dataset['classificacao_baixo_peso_test_X'],
    "classificacao_normal_peso_test_X": physionet2012_dataset['classificacao_normal_peso_test_X'],
    "classificacao_sobrepeso_test_X": physionet2012_dataset['classificacao_sobrepeso_test_X'],
    "classificacao_obesidade_1_test_X": physionet2012_dataset['classificacao_obesidade_1_test_X'],
    "classificacao_obesidade_2_test_X": physionet2012_dataset['classificacao_obesidade_2_test_X'],
    "classificacao_obesidade_3_test_X": physionet2012_dataset['classificacao_obesidade_3_test_X']
}
## calculate the mask to indicate the ground truth positions in test_X_ori, will be used by metric funcs to evaluate models
test_X_indicating_mask = []
test_X_ori = []
for i, j in zip(dataset_for_testing_ori.values(), dataset_for_testing.values()):
    test_X_indicating_mask.append(np.isnan(i) ^ np.isnan(j))
    test_X_ori.append(np.nan_to_num(i))   # metric functions do not accpet input with NaNs, hence fill NaNs with 0

### Inicialize the models

<h4> Inicialize new model</h4>

In [4]:
saits = SAITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-17 23:03:27 [INFO]: No given device, using default device: cpu
2025-03-17 23:03:27 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250317_T230327
2025-03-17 23:03:27 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250317_T230327/tensorboard
2025-03-17 23:03:27 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize existing model</h4>

In [4]:
saits = SAITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 19:17:38 [INFO]: No given device, using default device: cpu
2025-03-16 19:17:38 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-16 19:17:38 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize new model</h4>

In [10]:
brits = BRITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-01-29 19:12:21 [INFO]: No given device, using default device: cuda
2025-01-29 19:12:21 [INFO]: Model files will be saved to tutorial_results/imputation/brits/20250129_T191221
2025-01-29 19:12:21 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/brits/20250129_T191221/tensorboard
2025-01-29 19:12:21 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 255,344


<h4> Inicialize existing model</h4>

In [5]:
brits = BRITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 19:17:44 [INFO]: No given device, using default device: cpu
2025-03-16 19:17:44 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-16 19:17:44 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


<h4> Inicialize new model</h4>

In [ ]:
us_gan = USGAN(
    n_steps= physionet2012_dataset['n_steps'],
    n_features= physionet2012_dataset['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/us_gan",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-10 23:06:16 [INFO]: No given device, using default device: cuda
2025-02-10 23:06:16 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-10 23:06:16 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


<h4> Inicialize existing model</h4>

In [6]:
us_gan = USGAN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 19:17:49 [INFO]: No given device, using default device: cpu
2025-03-16 19:17:49 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-16 19:17:49 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


<h4> Inicialize new model</h4>

In [ ]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/gp_vae",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)


2025-02-10 23:06:18 [INFO]: No given device, using default device: cuda
2025-02-10 23:06:18 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-10 23:06:18 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


<h4> Inicialize existing model</h4>

In [7]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 19:17:54 [INFO]: No given device, using default device: cpu
2025-03-16 19:17:54 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-16 19:17:54 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


<h4> Inicialize new model</h4>

In [ ]:
mrnn = MRNN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/mrnn",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
) 


2025-02-10 23:06:21 [INFO]: No given device, using default device: cuda
2025-02-10 23:06:21 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-10 23:06:21 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


<h4> Inicialize existing model</h4>

In [8]:
mrnn = MRNN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-16 19:17:59 [INFO]: No given device, using default device: cpu
2025-03-16 19:17:59 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-16 19:17:59 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


### Train the models

In [5]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
saits.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-03-17 23:03:50 [INFO]: Epoch 001 - training loss (MSE): 0.7958, validation MSE: 6.7265
2025-03-17 23:03:59 [INFO]: Epoch 002 - training loss (MSE): 0.5808, validation MSE: 6.6878
2025-03-17 23:04:09 [INFO]: Epoch 003 - training loss (MSE): 0.5045, validation MSE: 6.6668
2025-03-17 23:04:18 [INFO]: Epoch 004 - training loss (MSE): 0.4531, validation MSE: 6.6624
2025-03-17 23:04:28 [INFO]: Epoch 005 - training loss (MSE): 0.4467, validation MSE: 6.6489
2025-03-17 23:04:38 [INFO]: Epoch 006 - training loss (MSE): 0.4151, validation MSE: 6.6447
2025-03-17 23:04:47 [INFO]: Epoch 007 - training loss (MSE): 0.4244, validation MSE: 6.6355
2025-03-17 23:04:57 [INFO]: Epoch 008 - training loss (MSE): 0.4267, validation MSE: 6.7094
2025-03-17 23:05:06 [INFO]: Epoch 009 - training loss (MSE): 0.3915, validation MSE: 6.6419
2025-03-17 23:05:16 [INFO]: Epoch 010 - training loss (MSE): 0.4287, validation MSE: 6.6349
2025-03-17 23:05:16 [INFO]: Finished training. The best model is from epoch#10.


In [11]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
brits.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-01-29 19:13:29 [INFO]: Epoch 001 - training loss: 0.8504, validation loss: 0.2889
2025-01-29 19:13:29 [INFO]: Saved the model to tutorial_results/imputation/brits/20250129_T191221/BRITS_epoch1_loss0.28890601024031637.pypots
2025-01-29 19:14:17 [INFO]: Epoch 002 - training loss: 0.5925, validation loss: 0.2504
2025-01-29 19:14:17 [INFO]: Saved the model to tutorial_results/imputation/brits/20250129_T191221/BRITS_epoch2_loss0.25044039537509283.pypots
2025-01-29 19:15:06 [INFO]: Epoch 003 - training loss: 0.5444, validation loss: 0.2404
2025-01-29 19:15:06 [INFO]: Saved the model to tutorial_results/imputation/brits/20250129_T191221/BRITS_epoch3_loss0.24043585633238157.pypots
2025-01-29 19:15:55 [INFO]: Epoch 004 - training loss: 0.5230, validation loss: 0.2346
2025-01-29 19:15:55 [INFO]: Saved the model to tutorial_results/imputation/brits/20250129_T191221/BRITS_epoch4_loss0.23458528518676758.pypots
2025-01-29 19:16:43 [INFO]: Epoch 005 - training loss: 0.5095, validation loss: 0.23

In [9]:
saits.load("tutorial_results/imputation/saits/standard/SAITS.pypots")

2025-03-16 19:18:03 [INFO]: Model loaded successfully from tutorial_results/imputation/saits/standard/SAITS.pypots


In [10]:
brits.load("tutorial_results/imputation/brits/standard/BRITS.pypots")

2025-03-16 19:18:06 [INFO]: Model loaded successfully from tutorial_results/imputation/brits/standard/BRITS.pypots


In [11]:
us_gan.load("tutorial_results/imputation/us_gan/standard/USGAN.pypots") 

2025-03-16 19:18:07 [INFO]: Model loaded successfully from tutorial_results/imputation/us_gan/standard/USGAN.pypots


In [12]:
gp_vae.load("tutorial_results/imputation/gp_vae/standard/GPVAE.pypots")

2025-03-16 19:18:09 [INFO]: Model loaded successfully from tutorial_results/imputation/gp_vae/standard/GPVAE.pypots


In [13]:
mrnn.load("tutorial_results/imputation/mrnn/standard/MRNN.pypots")

2025-03-16 19:18:12 [INFO]: Model loaded successfully from tutorial_results/imputation/mrnn/standard/MRNN.pypots


### The testing stage

In [6]:
# the testing stage, impute the originally-missing values and artificially-missing values in the test set
saits_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   saits_results = saits.predict(_dict)
   saits_imputation.append(saits_results["imputation"])   

In [12]:
# the testing stage, impute the originally-missing values and artificially-missing values in the test set
brits_imputation = []
for value in dataset_for_testing.values():
    _dict = {'X':value}
    brits_results = brits.predict(_dict)
    brits_imputation.append(brits_results["imputation"])

In [44]:
usgan_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   usgan_results = us_gan.predict(_dict)
   usgan_imputation.append(usgan_results["imputation"])


In [45]:
gpvae_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   gpvae_results = gp_vae.predict(_dict)
   gpvae_imputation.append(gpvae_results["imputation"])

In [46]:
mrnn_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   mrnn_results = mrnn.predict(_dict)
   mrnn_imputation.append(mrnn_results["imputation"])


### Calculate mean absolute error

In [7]:
testing_mae_saits = []
for i in range(len(saits_imputation)):
    testing_mae_saits.append(calc_mae(saits_imputation[i], test_X_ori[i], test_X_indicating_mask[i]))



In [13]:
testing_mae_brits = []
for i in range(len(brits_imputation)):
    testing_mae_brits.append(calc_mae(brits_imputation[i], test_X_ori[i], test_X_indicating_mask[i]))


In [47]:
testing_mae_usgan = []
for i in range(len(usgan_imputation)):
    testing_mae_usgan.append(calc_mae(usgan_imputation[i], test_X_ori[i], test_X_indicating_mask[i]))

In [48]:
for i in range(len(gpvae_imputation)):
    gpvae_imputation[i] = gpvae_imputation[i].reshape(len(gpvae_imputation[i]), 48, 37)

In [49]:
testing_mae_gpvae = []
for i in range(len(gpvae_imputation)):
    testing_mae_gpvae.append(calc_mae(gpvae_imputation[i], test_X_ori[i], test_X_indicating_mask[i]))

In [50]:
testing_mae_mrnn = []
for i in range(len(mrnn_imputation)):
    testing_mae_mrnn.append(calc_mae(mrnn_imputation[i], test_X_ori[i], test_X_indicating_mask[i]))

In [8]:
subgroups = ["General", "Female", "Male", "Undefined Gender", "+65", "-65", "ICUType 1", "ICUType 2", "ICUType 3", "ICUType 4", "Undefined classification", "Low Weight", "Normal Weight", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"]

In [9]:
print("SAITS - MAE")
print("-------------")
for i in range(len(subgroups)):
    print(subgroups[i], ":" ,testing_mae_saits[i])

SAITS - MAE
-------------
General : 0.27077346305383765
Female : 0.27205880564407836
Male : 0.2749153674534257
Undefined Gender : 0.20100186339744913
+65 : 0.2667339001575028
-65 : 0.27959884451842193
ICUType 1 : 0.28883602706039174
ICUType 2 : 0.2429101299201823
ICUType 3 : 0.2876700677245509
ICUType 4 : 0.2775607868042399
Undefined classification : 0.2814047523152634
Low Weight : 0.2901282325497619
Normal Weight : 0.25461907386388605
Overweight : 0.25833916107879573
Obesity 1 : 0.2786189352851397
Obesity 2 : 0.2568809183361219
Obesity 3 : 0.2736831699961278


In [15]:
print("BRITS - MAE")
print("-------------")
for i in range(len(subgroups)):
    print(subgroups[i], ":" ,testing_mae_brits[i])

BRITS - MAE
-------------
General : 0.20607649494742675
Female : 0.20614220432360486
Male : 0.21246684469801871
Undefined Gender : 0.18272824533494633
+65 : 0.20458278834580312
-65 : 0.21627359080249212
ICUType 1 : 0.220299073686516
ICUType 2 : 0.1808477823544893
ICUType 3 : 0.22441586013095977
ICUType 4 : 0.20966047499437565
Undefined classification : 0.21801499897060275
Low Weight : 0.24078274437854238
Normal Weight : 0.195872946252596
Overweight : 0.19804599357933922
Obesity 1 : 0.19122453585431926
Obesity 2 : 0.2035281786613216
Obesity 3 : 0.2143222467284282


In [51]:
print("USGAN - MAE")
print("-------------")
for i in range(len(subgroups)):
    print(subgroups[i], ":" ,testing_mae_usgan[i])


USGAN - MAE
-------------
General : 0.2725404672962992
Female : 0.27515345338175035
Male : 0.2759156782410606
Undefined Gender : 0.22719787038605313
+65 : 0.2703022819200602
-65 : 0.28051891797636347
ICUType 1 : 0.29337843510987965
ICUType 2 : 0.23865748003371157
ICUType 3 : 0.29412443520709886
ICUType 4 : 0.2800859210695823
Undefined classification : 0.2877627249325782
Low Weight : 0.30363618669583015
Normal Weight : 0.2588177315143253
Overweight : 0.2544338181108165
Obesity 1 : 0.2758376945836133
Obesity 2 : 0.2619401389587843
Obesity 3 : 0.2693813004105881


In [52]:
print("MRNN - MAE")
print("-------------")
for i in range(len(subgroups)):
    print(subgroups[i], ":" ,testing_mae_mrnn[i])


MRNN - MAE
-------------
General : 0.6713145721873255
Female : 0.6886168215267683
Male : 0.6593698528405196
Undefined Gender : 0.5788306694697075
+65 : 0.6680155576961296
-65 : 0.6851731084848083
ICUType 1 : 0.7000602620167663
ICUType 2 : 0.5960496586403451
ICUType 3 : 0.7305747176870081
ICUType 4 : 0.6660171755623768
Undefined classification : 0.6998902963878201
Low Weight : 0.7413824045826151
Normal Weight : 0.6484704845721746
Overweight : 0.6258698324694673
Obesity 1 : 0.6399939492108968
Obesity 2 : 0.6628144867515143
Obesity 3 : 0.7372770171765964


In [55]:

print("GPVAE - MAE")
print("-------------")
for i in range(len(subgroups)):
    print(subgroups[i], ":" ,testing_mae_gpvae[i])

GPVAE - MAE
-------------
General : 0.454358819819243
Female : 0.45964569560182006
Male : 0.4539033170525852
Undefined Gender : 0.3952713921281636
+65 : 0.45158590664180026
-65 : 0.4686480177725102
ICUType 1 : 0.4759892460652797
ICUType 2 : 0.39423516109862916
ICUType 3 : 0.49689965362643795
ICUType 4 : 0.46005345089771515
Undefined classification : 0.47611340106275457
Low Weight : 0.5194780203246361
Normal Weight : 0.4300186668030726
Overweight : 0.4300886671089587
Obesity 1 : 0.45186196738172746
Obesity 2 : 0.4501944866258905
Obesity 3 : 0.4661876578901662
